In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
sys.path.append('/home/zhenyujin/qso_fitting/')
import h5py
from qso_fitting.analysis.inference_test import run_inference_test, compute_importance_weights, C_ge

ModuleNotFoundError: No module named 'pypeit'

In [ ]:
x_size = 3.5
dpi_value = 200

plt_params = {'legend.fontsize': 7,
              'legend.frameon': False,
              'axes.labelsize': 8,
              'axes.titlesize': 8,
              'figure.titlesize': 7,
              'xtick.labelsize': 7,
              'ytick.labelsize': 7,
              'lines.linewidth': .7,
              'lines.markersize': 2.3,
              'lines.markeredgewidth': .9,
              'errorbar.capsize': 2,
              'font.family': 'serif',
              # 'text.usetex': True,
              'xtick.minor.visible': True,
              }
plt.rcParams.update(plt_params)

out_path = '/mnt/quasar2/zhenyujin/igm_emulator/hmc/hmc_results/'

In [ ]:
def coverage_overplot(test_nums):
    alpha_vec = np.concatenate((np.linspace(0.00, 0.994, num=100), np.linspace(0.995, 1.0, num=51)))
    inference_fig = plt.figure(figsize=(x_size, x_size * .9), constrained_layout=True,
                                   dpi=dpi_value,
                                   )
    grid = inference_fig.add_gridspec(nrows=1, ncols=1)
    skew_ax = inference_fig.add_subplot(grid[0])
    for i, num in enumerate(test_nums):
        file_name = f'z54_train_55_bin59_seed_11_inference_100_gaussian_mocks_emulator_seed_36_samples_1000_chains_4_nn_err_prop_True{num}'
        f = h5py.File(out_path + file_name +'.hdf5','r+')    
        log_prob_x = f['log_prob_x']
        true_log_prob_x = f['true_log_prob_x']
        coverage_gauss, coverage_gauss_lo, coverage_gauss_hi = run_inference_test(alpha_vec, log_prob_x,
                                                                                  true_log_prob_x,
                                                                                  title=num, show=False)
        skew_ax.plot(alpha_vec, coverage_gauss, color=f'C{i}', linestyle='solid', label='inference test points',
                     zorder=10)
        skew_ax.fill_between(alpha_vec, coverage_gauss_lo, coverage_gauss_hi, facecolor=f'C{i}', alpha=0.8, zorder=3)
    x_vec = np.linspace(0.0, 1.0, 11)
    skew_ax.plot(x_vec, x_vec, linewidth=1.5, color='red', linestyle=(0, (5, 10)), zorder=20,
                     label='inferred model')
    skew_ax.set_xlim((-0.01, 1.01))
    skew_ax.set_ylim((-0.01, 1.01))
    skew_ax.set_xlabel(r'$P_{{\rm true}}$', fontsize=16)
    skew_ax.set_ylabel(r'$P_{{\rm inf}}$', fontsize=16)
